In [1]:
partition = 100

In [2]:
import sys
import os

In [3]:
n_tree_values = [5, 10, 20, 50, 100, 150, 200, 300]
tree_depth_values = [1, 2, 3, 4, 5, 6, 7]
hidden_dim = [1024, 768]
batch_size_values = [256, 512, 1000]
tree_feature_rates = [0.1, 0.2, 0.3, 0.4, 0.5]
feat_dropouts = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
lrs = [0.0001, 0.001, 0.01]

amount = len(n_tree_values) * len(tree_depth_values) * len(lrs)
amount

120

In [4]:
import os
import nbformat as nbf

# Hyperparameter grid
n_tree_values = [10, 20, 50, 100, 150, 200]
tree_depth_values = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
lrs = [0.001, 0.01]

# Base directory
base_dir = "experiments"
os.makedirs(base_dir, exist_ok=True)

# Function to create the notebook
def create_notebook(n_trees, t_depth, lr):
    nb = nbf.v4.new_notebook()

    # Cell 1: partition
    nb.cells.append(nbf.v4.new_code_cell("partition = 100"))

    # Cell 2: imports
    nb.cells.append(nbf.v4.new_code_cell(
        "import sys\nfrom itertools import product"
    ))

    nb.cells.append(nbf.v4.new_code_cell(
        """import os\nsys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))\nfrom train import main"""
    ))

    nb.cells.append(nbf.v4.new_code_cell(
        """experiment_dir = os.getcwd()\nos.chdir(os.path.abspath(os.path.join(os.getcwd(), "../..")))"""
    ))

    # Cell 3: hyperparameter grid (customized per loop)
    cell3_code = f"""n_tree_values = [{n_trees}]
tree_depth_values = [{t_depth}]
hidden_dim = [1024, 768]
batch_size_values = [256, 512]
tree_feature_rates = [0.1, 0.2]
feat_dropouts = [0.0, 0.1, 0.2]
lrs = [{lr}]
"""
    nb.cells.append(nbf.v4.new_code_cell(cell3_code))

    # Cell 4: search and training logic
    cell4_code = """best_score = 0
best_config = {}

for n_tree, t_depth, hd, batch_size, tfr, do, lr in product(n_tree_values, tree_depth_values, hidden_dim, batch_size_values, tree_feature_rates, feat_dropouts, lrs):
    print(f"Running: n_tree={n_tree}, t_depth={t_depth}, hd={hd}, batch_size={batch_size} tree_feature_rates={tfr}, feat_dropouts={do}, hd={hd}, lrs={lr}")
    results_dir = os.path.join(experiment_dir, "results")
    os.makedirs(results_dir, exist_ok=True)

    sys.argv = [
        'train.py',
        '-dataset', f'gtd{partition}',
        '-n_class', '30',
        '-gpuid', '0',
        '-n_tree', str(n_tree),
        '-tree_depth', str(t_depth),
        '-batch_size', str(batch_size),
        '-hidden_dim', str(hd),
        '-epochs', '300',
        '-verbose', '0',
        '-tree_feature_rate', str(tfr),
        '-feat_dropout', str(do),
        '-lr', str(lr),
        '-jointly_training',
        '-searching', '1',
        '-results_dir', results_dir
    ]

    complete = main()
    print(complete)

    
    result_file = os.path.join(results_dir, f"result_gtd{partition}.txt")
    try:
        with open(result_file, "r") as f:
            lines = f.readlines()
            for line in lines:
                if "Best Accuracy" in line:
                    acc = float(line.split()[2])
                    if acc > best_score:
                        best_score = acc
                        best_config = {
                            'n_tree': n_tree,
                            'tree_depth': t_depth,
                            'batch_size': batch_size,
                            'hidden_dim': hd,
                            'tree_feature_rate': tfr,
                            'feat_dropout': do,
                            'lr': lr
                        }
    except FileNotFoundError:
        print(f"ERROR: Result file not found: {result_file}")
print("Best hyperparameter configuration:")
print(best_config)
print(f"Best accuracy: {best_score}")

try:
    with open(result_file, "a") as f:
        for k, v in best_config.items():
            f.write(f"{k}: {v}, ")
except Exception as e:
    print(f"ERROR: Failed to write best_config to result file: {e}")
"""
    nb.cells.append(nbf.v4.new_code_cell(cell4_code))
    return nb

# Folder creation + notebook generation
count = 0
for n_trees in n_tree_values:
    for depth in tree_depth_values:
        for lr in lrs:
            lr_str = str(lr).replace('.', 'p')
            folder_name = f"trees_{n_trees}_depth_{depth}_lr_{lr_str}"
            folder_path = os.path.join(base_dir, folder_name)
            os.makedirs(folder_path, exist_ok=True)

            # Create notebook and save
            notebook = create_notebook(n_trees, depth, lr)
            notebook_path = os.path.join(folder_path, "gtd100.ipynb")
            with open(notebook_path, "w") as f:
                nbf.write(notebook, f)
            count += 1

print(f"{count} folders and notebooks created successfully.")


120 folders and notebooks created successfully.
